## Modeling with Julia with Ipopt

Anything with an _a_ reference relates to 'our' team. 
Anything with a _b_ reference relates to 'their' team.

In [31]:
using PyCall
println(PyCall.python)

/Users/lauraroettges/.julia/conda/3/x86_64/bin/python


In [32]:
using PyCall
pyimport_conda("pandas", "pandas")

PyObject <module 'pandas' from '/Users/lauraroettges/.julia/conda/3/x86_64/lib/python3.10/site-packages/pandas/__init__.py'>

In [33]:
using PyCall
pd = pyimport("pandas")

PyObject <module 'pandas' from '/Users/lauraroettges/.julia/conda/3/x86_64/lib/python3.10/site-packages/pandas/__init__.py'>

In [34]:
using DataFrames, PyCall

#Define Data
p_a1 = 0.000518  # (precomputed) probability of score when 5v5
p_b1 = 0.000518  # (precomputed) probability of score when 5v5

p_a2 = 0.00310   # (precomputed) probability that we score during a given second in 6v5 play
p_b2 = 0.00851   # (precomputed) probability that the opponent scores during a given second in 6v5 play

T_big = 1200 # number of seconds remaining in the game at start of period 3 (20 minutes)

comb_df = pd.read_pickle("../precompute_combinations/precomputed_combinations_df.pkl")
println(comb_df.head())

function choose(n, k)
    #found out through trial and error n must be less than 1200 can not = 1200
    index = n * 1201 + k
    #because of julia need to add 1 to index
    index += 1
    return comb_df.loc[index]["n_choose_k"]
end


#A_0 = [] #TODO: stochastic set -  Our team’s current score (before decision)
#B_0 = [] #TODO: stochastic set -  Their team’s current score (before decision)

scenarios = [:tied, :behind_one, :behind_two]
strategies = [:pull_goalie, :play_normal]

a_0 = Dict(zip(scenarios, [0, 0, 0]))
b_0 = Dict(zip(scenarios, [0, 1, 2]))

#going to start by assuming we are in the behind 1 scenario
A_0 = a_0[:behind_one]
B_0 = b_0[:behind_one]

M = 3599 #be the maximum possible value of d (if their team scored every second of the game and our team never scored)


PyObject            n  k n_choose_k
new_index                 
0          0  0          1
1201       1  0          1
1202       1  1          1
2402       2  0          1
2403       2  1          2


3599

$\mathbb{P}[\mathscr{E}_i] = \mathbb{P}[(G_{a1} == g_{a1}) \& (G_{a2} == g_{a2}) \& (G_{b1} == g_{b1}) \& (G_{b2} = g_{b2})]$ \\
    $ = {s \choose g_{a1}}(p_{a1})^{g_{a1}}(1 - p_{a1})^{s - g_{a1}} \cdot {s \choose g_{b1}}(p_{b1})^{g_{b1}}(1 - p_{b1})^{s - g_{b1}} \cdot {T - s \choose g_{a2}}(p_{a2})^{g_{a2}}(1 - p_{a2})^{T - s - g_{a2}} \cdot {T - s \choose g_{b2}}(p_{b2})^{g_{b2}}(1 - p_{b2})^{T - s - g_{b2}}$

Need to break this down:
${s \choose g_{a1}}(p_{a1})^{g_{a1}}(1 - p_{a1})^{s - g_{a1}} \cdot {s \choose g_{b1}}(p_{b1})^{g_{b1}}(1 - p_{b1})^{s - g_{b1}} \cdot {T - s \choose g_{a2}}(p_{a2})^{g_{a2}}(1 - p_{a2})^{T - s - g_{a2}} \cdot {T - s \choose g_{b2}}(p_{b2})^{g_{b2}}(1 - p_{b2})^{T - s - g_{b2}}$

In [35]:
function probability_of_event(s, T, prob_a1, prob_b1, prob_a2, prob_b2, ga_before, gb_before, ga2, gb2)
    
    #s = number of seconds in epoch 1 (5v5)
    #T = number of seconds in epoch 2 (6v5)
    #prob_a1 = probability of scoring in epoch 1 (5v5)
    #prob_b1 = probability of opponent scoring in epoch 1 (5v5)
    #prob_a2 = probability of scoring in epoch 2 (6v5)
    #prob_b2 = probability of opponent scoring in epoch 2 (6v5)
    #ga_before = number of goals our team has before goalie pull (epoch 0 & 1)
    #gb_before = number of goals their team has before after goalie pull (epoch 0 & 1)
    #ga2 = number of goals our team has after goalie pull (epoch 2)
    #gb2= number of goals their team has after goalie pull (epoch 2)
    #ensure relevant values are integers
    s = Int(s)
    T = Int(T)
    ga_before = Int(ga_before)
    gb_before = Int(gb_before)
    ga2 = Int(ga2)
    gb2 = Int(gb2)

    #because indicies 
    calcd_prob = choose(s, ga_before) * prob_a1^ga_before * (1 - prob_a1)^(s - ga_before) * choose(s, gb_before) * prob_b1^gb_before * (1 - prob_b1)^(s - gb_before) * choose(T-s, ga2) * prob_a2^ga2 * (1 - prob_a2)^(T - s - ga2) * choose(T-s, gb2) * prob_b2^gb2 * (1 - prob_b2)^(T - s - gb2)
    return calcd_prob
end

probability_of_event (generic function with 1 method)

In [36]:
# Generate all possible states
# Some extra notes:
#important they must all be ints and they must all be >= 0
# 0 =< ga1 <= s #assuming we can't score more than the number of seconds in epoch 1 (5v5) 
# 0 =< gb1 <= s #assuming we can't score more than the number of seconds in epoch 1 (5v5) 

# 0 =< ga2 <= T-s #assuming we can't score more than the number of seconds in epoch 2 (6v5)
# 0 =< gb2 <= T-s #assuming we can't score more than the number of seconds in epoch 2 (6v5) 

#if we want to make s interval a minute for computational tractability could do T = 5 * 60 = 300 (5 minutes)

#will start with just 2 minutes
T_big = 5 * 60  #2 minutes

T_big = T_big / 10 #I want S to be 10 second increments
# then also need to update probabilities by a factor of 10 
p_a1 = p_a1*10
p_b1 = p_b1*10

p_a2 = p_a2*10
p_b2 = p_b2*10
states = []
for s in 1:T_big
    for ga1 in 0:s
        for gb1 in 0:s
            for ga2 in 0:T-s
                for gb2 in 0:T-s
                    push!(states, (s, ga1, gb1, ga2, gb2))
                end
            end
        end
    end
end


In [8]:

# Generate probabilities
probabilities = Dict()
#effectively could precompute w 
win_probs = Dict()
for state in states
    s, ga1, gb1, ga2, gb2 = state
    success = ga1 + ga2 >= gb1 + gb2
    #println("state: ", state)
    prob = probability_of_event(s, T, p_a1, p_b1, p_a2, p_b2, ga1, gb1, ga2, gb2)
    #println("prob: ", prob)
    probabilities[state] = prob
    if success == true
        win_probs[state] = 1
    else 
        win_probs[state] = 0
    end
end
;

In [ ]:
# I want to save this dictionary to a file so I can load it later
using Serialization

# Save the dictionary to a file
open("probabilities_dict.jls", "w") do io
    serialize(io, probabilities)
end
open("success_state_dict.jls", "w") do io
    serialize(io, win_probs)
end

#may also want to save as a JSON file as back up 
using JSON

# Save the dictionary to a file
open("probabilities_dict_backup.json", "w") do io
    JSON.print(io, probabilities)
end
open("success_state_dict_backup.json", "w") do io
    JSON.print(io, win_probs)
end

In [ ]:
using Serialization
# Load the dictionary from the file
probabilities_loaded = open("probabilities_dict.jls", "r") do io
    deserialize(io)
end

In [11]:
using JuMP, HiGHS, MathOptInterface

# I think M can now be 359 (since we are now in 10 second increments)
# M = 359 # I will start with this commented though just in case

model = Model()

# Variables
# assuming we don't choose s = 0 (i.e. pull the goalie immediately - TODO: check w/ Ryan) 
@variable(model, 1 <= s <= T_big, Int)      # s: how many seconds to wait before pulling the goalie 
@variable(model, y, Bin)          # y: indicator variable for final score difference
                                  # y = 1 => d is positive (indicating a win or tie), y = 0 => or negative (indicating a loss)
# @variable(model, d, Int)          # d: final score difference (our team - their team)

# Initialize an empty dictionary to store the variables
w = Dict{Tuple{Int, Int, Int, Int, Int}, JuMP.VariableRef}()

for s in 1:T_big
    for ga1 in 0:s
        for gb1 in 0:s
            for ga2 in 0:T_big-s
                for gb2 in 0:T_big-s
                    # Set up variable w for each scenario

                    w[(s, ga1, gb1, ga2, gb2)] = @variable(model, base_name="w_$(s)_$(ga1)_$(gb1)_$(ga2)_$(gb2)")
                    # Set up w[i] < probabilities of state 
                    # (s, T, p_a1, p_b1, p_a2, p_b2, ga1, gb1, ga2, gb2)
                    curr_prob = probability_of_event(s, T_big, p_a1, p_b1, p_a2, p_b2, ga1, gb1, ga2, gb2)
                    #@expression(model, curr_prob, probability_of_event(s, T, p_a1, p_b1, p_a2, p_b2, ga1, gb1, ga2, gb2))
                    # Convert curr_prob to a Julia type if it is a PyCall.jlwrap object
                    curr_prob = convert(Float64, curr_prob)
                    @constraint(model, w[(s, ga1, gb1, ga2, gb2)] <= curr_prob)
                    d = (A_0 + ga1 + ga2) + 1 - (B_0 + gb1 + gb2)  # Calculate the score difference
                    @constraint(model, w[(s, ga1, gb1, ga2, gb2)] >= 0)
                    @constraint(model, w[(s, ga1, gb1, ga2, gb2)] >= d)
                    @constraint(model, w[(s, ga1, gb1, ga2, gb2)] <= M - (M * y))
                    @constraint(model, w[(s, ga1, gb1, ga2, gb2)] <= d + M * y)
                    #@constraint(model, w[(s, ga1, gb1, ga2, gb2)] <= M * y)
                    #@constraint(model, w[(s, ga1, gb1, ga2, gb2)] <= d + M * (1-y))
                end
            end
        end
    end
end

# Objective
@objective(model, Max, sum(w[key] for key in keys(w)))
;


In [40]:
using JuMP, HiGHS, MathOptInterface

model = Model()
# I think M can now be 359 (since we are now in 10 second increments)
M = 359 # I will start with this commented though just in case

#Variables
#assuming we don't choose s = 0 (i.e. pull the goalie immediately - TODO: check w/ Ryan) 
@variable(model, 0 <= s <= T, Int)      # s: how many seconds to wait before pulling the goalie 
@variable(model, w[1:length(states)] >= 0) # w: probability of each scenario   
@variable(model, y[1:length(states)], Bin) # y: indicator variable for final score difference
                                # y = 0 => d is positive (indicating a win or tie), y = 1 => or negative (indicating a loss) 

#Constraints
# Define constraints based on states (s, ga1, gb1, ga2, gb2)
for (i, state) in enumerate(states)
    s_for_now, ga1, gb1, ga2, gb2 = state
    d = (A_0 +ga1 + ga2) +1 - (B_0 + gb1 + gb2)  # Calculate the score difference
    #println("d: ", d)
    #if d >= 0
    #    y = 0
    #else
    #    y = 1
    #end

    #need to make sure y is appropriately defined
    @constraint(model, M*y[i] <= M - d)

    #@constraint(model, w[i] <= convert(Float64, probabilities[state]))
    #if I don't relate it to probabiliies and only d 
    @constraint(model, w[i] >= d) 

    #ok thoughts cant say w is greater than or equal to 0 and less than or equal to d because d could be negative
    #that will make this infeasible

    @constraint(model, w[i] <=  M - (M * y[i]))
    @constraint(model, w[i] <= d + M * y[i])
    
    #need to actually incorporate s appropriately - which I am not doing now, it's a variable I am never using.. 

end

# Objective
@objective(model, Max, sum(w[i] for i in 1:length(states)))


w[1] + w[2] + w[3] + w[4] + w[5] + w[6] + w[7] + w[8] + w[9] + w[10] + w[11] + w[12] + w[13] + w[14] + w[15] + w[16] + w[17] + w[18] + w[19] + w[20] + w[21] + w[22] + w[23] + w[24] + w[25] + w[26] + w[27] + w[28] + w[29] + w[30] + [[...1117459 terms omitted...]] + w[1117490] + w[1117491] + w[1117492] + w[1117493] + w[1117494] + w[1117495] + w[1117496] + w[1117497] + w[1117498] + w[1117499] + w[1117500] + w[1117501] + w[1117502] + w[1117503] + w[1117504] + w[1117505] + w[1117506] + w[1117507] + w[1117508] + w[1117509] + w[1117510] + w[1117511] + w[1117512] + w[1117513] + w[1117514] + w[1117515] + w[1117516] + w[1117517] + w[1117518] + w[1117519]

In [41]:
using JuMP, HiGHS, MathOptInterface


#set solver
set_optimizer(model, HiGHS.Optimizer)


# Print constraints
#println("Constraints:")
#for c in all_constraints(model, include_variable_in_set_constraints=true)
#    # Get lower and upper bounds for the constraint
#    try
#        lb = MathOptInterface.get(model, MathOptInterface.LowerBound, c)
#        ub = MathOptInterface.get(model, MathOptInterface.UpperBound, c)
#        if ub < lb
#            println("Conflict detected")
#            println(c)
#        end
#    catch e
#        # Handle cases where bounds are not available
#        println("No bounds available for constraint: ", c)
#    end
#end


println("Number of constraints: ", num_constraints(model, count_variable_in_set_constraints=true))
#print the lengthe of w
println("Length of w: ", length(w))

# Solve the model
@time(optimize!(model))

# Print the results
println("Objective value: ", objective_value(model))
println("pull out the goalie at time s: ", value(s))


Number of constraints: 6705117
Length of w: 1117519
Running HiGHS 1.7.2 (git hash: 5ce7a2753): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 4e+02]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 3e+01]
  RHS    [1e+00, 4e+02]
Presolving model
1161264 rows, 624377 cols, 1248754 nonzeros  2s
87490 rows, 78740 cols, 157480 nonzeros  3s
69990 rows, 62990 cols, 125980 nonzeros  3s
55990 rows, 50390 cols, 100780 nonzeros  3s
44790 rows, 40310 cols, 80620 nonzeros  3s
35830 rows, 32246 cols, 64492 nonzeros  3s
28662 rows, 25795 cols, 51590 nonzeros  4s
22928 rows, 20635 cols, 41270 nonzeros  4s
18342 rows, 16507 cols, 33014 nonzeros  4s
14672 rows, 13204 cols, 26408 nonzeros  4s
11736 rows, 10562 cols, 21124 nonzeros  4s
9388 rows, 8387 cols, 16774 nonzeros  4s
7386 rows, 6385 cols, 12770 nonzeros  4s
5384 rows, 4383 cols, 8766 nonzeros  4s
3382 rows, 2381 cols, 4762 nonzeros  4s
1380 rows, 690 cols, 1380 nonzeros  4s
0 rows, 0 cols, 0 nonzeros  4s
Presolve: 

In [ ]:
#so far just getting pull them at the very beginning which I know is wrong for 5 minutes 
#I think I need to factor in probability still not just max(d, 0)
#also s is not actually making an appearance in my model

In [ ]:
##Code for debugging
## Print the DataFrame's index and columns for debugging
#println("DataFrame Index: ", comb_df.index)
#println("DataFrame Columns: ", comb_df.columns)
#println("Number of Rows: ", comb_df.shape[1])
#
## Access a specific row
#row = comb_df.loc[1202]
#println("this row put in 1202:", row)
#row = comb_df.loc[1203]
#println("next row put in 1203: ", row)
#row = comb_df.loc[2403]
#println("3rd row put in 2403: ",row)
#
## Access a specific element (ensure the index exists and the column name is correct)
#println("this other thing: ",comb_df.loc[1202]["n_choose_k"])
#
#test_mult = comb_df.loc[1202]["n_choose_k"] * .05
#println("test_mult: ", test_mult)